In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

## Payoff Function

In [84]:
def payoff(paths, Nsim, K, Barrier, ratio):
    final_prices = paths[:, :, -1] #S_T
    above_strike = np.all(final_prices >= K, axis=1)
    barrier_event = np.any(paths.min(axis=2) <= Barrier, axis=1)

    # No barrier event or (barrier event and above strike)
    cash_payout = np.where(np.logical_or(np.logical_not(barrier_event), np.logical_and(barrier_event,  above_strike)), 1105, 0)

    # barrier event and below strike
    percent_change = final_prices / K # Based on final strike price
    min_loc = np.argmin(percent_change, axis=1) # Finding the worst performing underlying
    j = np.indices(min_loc.shape)
    payouts = ratio[min_loc] * final_prices[j, min_loc] + 105 # certificate to stock ratio * final price of the stock + coupon payment
    stock_payout = np.where(np.logical_and(barrier_event,  np.logical_not(above_strike)), payouts, 0)

    return np.sum(np.concatenate((cash_payout.reshape(1, Nsim), stock_payout.reshape(1, Nsim))), axis=0)

## Plotting

In [6]:
def plot_simulations(price_hist, Nsim, sim_paths, stock=0):
    """plots simulations for chosen stock

    Args:
        price_hist (df): df of historical prices
        sim_paths (list): simulated price paths
        stock (int, optional): index of stock. Defaults to 0.
    """
    # df of historical prices
    price_hist = price_hist.to_numpy()[:, [0]].reshape(-1)
    price_hist = np.vstack([price_hist]*Nsim)
    price_hist = pd.DataFrame(price_hist).transpose()

    sim_paths = pd.DataFrame(sim_paths[:, stock, :]).transpose()
    sim_paths = pd.concat((price_hist, sim_paths))
    sim_paths = sim_paths.reset_index(drop=True)
    fig = px.line(sim_paths)
    return fig

# Standard MC Simulations (GBM)

In [4]:
def multi_asset_GBM(S0, v, Sigma, delta_t, m, p):
    """GBM simulation of multiple asset paths

    Args:
        S0 (vector of start prices): vector of starting prices
        v (array): mu/dt
        Sigma (matrix): covariance matrix
        delta_t (float): delta between each time step
        m (int): number of days simulated
        p (int): number of assets
    """

    S = np.zeros(shape=(m, p))
    S[0] = S0

    z = np.random.multivariate_normal(mean=v*delta_t, cov=Sigma*delta_t, size=m)
    for step in range(1, m):
        S[step] = np.exp(np.log(S[step-1]) + z[step-1])
    return np.transpose(S)

## Simulations

In [5]:
def n_path_sim(Nsim, v, Sigma, T, dt, S0, sim_func):
    m = int(T/dt)
    p = len(S0)
    paths = np.zeros(shape=(Nsim, p, m))

    for i in range(Nsim):
        paths[i] = sim_func(S0, v, Sigma, dt, m, p)
    return paths
        

In [2]:
df = pd.read_csv("assets.csv")
df.head()

,Date,CVX,UNH,XOM
0,2021-11-02,113.830002,452.040009,64.820000
1,2021-11-03,113.010002,457.329987,63.930000
2,2021-11-04,113.510002,456.760010,64.410004
3,2021-11-05,114.739998,455.809998,65.019997
4,2021-11-08,115.150002,462.619995,65.720001


In [10]:
dt = 1/252 # Trading days
prices = df.copy()
n0 = prices.shape[0] # Number of data points
prices = prices.drop(columns='Date')
K = np.array([103.87, 413.05, 58.26])
Barrier = np.array([62.322, 247.830, 34.956])
ratio = np.array([9.6274, 2.4210, 17.1644])
log_prices = prices.apply(lambda x: np.log(x))
log_returns = log_prices.apply(lambda x: np.diff(x)) 

Nsim = 10
T = 1
S0 = prices.to_numpy()[-1]
v = (log_returns.mean()/dt).values
Sigma = (log_returns.cov()/dt).to_numpy()
np.random.seed(4518)

print(f"v: {v}\nSigma (covariance matrix): \n{Sigma}")

v: [0.44988248 0.1841302  0.52531442]
Sigma (covariance matrix): 
[[0.10409419 0.02103573 0.09854091]
 [0.02103573 0.05591201 0.021925  ]
 [0.09854091 0.021925   0.12306251]]


In [80]:
# simulate stock path prices
paths = n_path_sim(Nsim, v, Sigma, T, dt, S0, multi_asset_GBM)

In [85]:
final_values = payoff(paths)

In [82]:
final_values

array([1105., 1105., 1105., 1105., 1105., 1105., 1105., 1105., 1105.,
       1105.])

In [ ]:
final_values[final_values < 1000]

# Antithetic Variate Simulations (GBM)